# Plinder Tutorial
## Data Processing Pipeline
Loads Plinder systems into dictionary of numpy arrays


In [1]:
from omtra_pipelines.plinder_dataset.plinder_pipeline import *
import plinder.core.utils.config

/net/galaxy/home/koes/tjkatz/.conda/envs/omtra/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The script will download the data from the remote data directory if it is not found in the local cache directory. 

Plinder data can be found on the cluster in `/net/galaxy/home/koes/tjkatz/.local/share/plinder/2024-06/v2`.

In [2]:
cfg = plinder.core.get_config()
print(f"local cache directory: {cfg.data.plinder_dir}")
print(f"remote data directory: {cfg.data.plinder_remote}")

local cache directory: /net/galaxy/home/koes/tjkatz/.local/share/plinder/2024-06/v2
remote data directory: gs://plinder/2024-06/v2


The `SystemProcessor` class takes in an `atom_map` and optionall a `pocket_cutoff`, which by default is set to 5A. 

The `process_system` call will convert the receptor and any linked apo structures into `StructureData` objects, which include numpy arrays of the atom coordinates, atom names, residue ids, residue names, and chain ids. 

For each ligand, a pocket will be extracted and returned as a `StructureData` object (and optionally saved as a PDB file). 

The ligands will be returned as `LigandData` objects with coords, atom_types, atom_charges, bond_types, and bond_indices as numpy arrays. 

The entire system is returned as a dictionary with the following keys: 'receptor', 'ligands', 'pockets', 'apo_structures', 'entry_annotation', 'system_annotation'. Ligands and pockets are dictionaries indexed by the plinder ligand ID and apo structures are indexed by the plinder linked structure id. Entry annotation corresponds to the PDB entry-level annotation stored in plinder and the system annotation is that at the sytem level. 

In [14]:
system_processor = SystemProcessor(
    ligand_atom_map=["C", "H", "N", "O", "F", "P", "S", "Cl", "Br", "I", "Se"],
    npnde_atom_map=["C", "H", "N", "O", "F", "P", "S", "Cl", "Br", "I", "Se"], 
    pocket_cutoff=8.0)
result = system_processor.process_system(system_id="4agi__1__1.C__1.W", save_pockets=False)

2025-02-06 18:38:30,197 | plinder.core.utils.cpl.download_paths:24 | INFO : runtime succeeded: 0.00s
2025-02-06 18:38:30,198 | plinder.core.utils.cpl.download_paths:24 | INFO : runtime succeeded: 0.00s
2025-02-06 18:38:30,286 | plinder.core.utils.cpl.download_paths:24 | INFO : runtime succeeded: 0.00s
2025-02-06 18:38:30,287 | plinder.core.utils.cpl.download_paths:24 | INFO : runtime succeeded: 0.00s
2025-02-06 18:38:30,288 | plinder.core.index.utils:148 | INFO : loading entries from 1 zips
2025-02-06 18:38:30,298 | plinder.core.index.utils:163 | INFO : loaded 1 entries
2025-02-06 18:38:30,299 | plinder.core.index.utils.load_entries:24 | INFO : runtime succeeded: 0.10s
2025-02-06 18:38:30,505 | plinder.core.utils.cpl.download_paths:24 | INFO : runtime succeeded: 0.09s
2025-02-06 18:38:30,953 | plinder.core.scores.links.query_links:24 | INFO : runtime succeeded: 0.65s
2025-02-06 18:38:33,720 | plinder.core.utils.cpl.download_paths:24 | INFO : runtime succeeded: 0.00s
2025-02-06 18:38:33

In [15]:
print(result.keys())

dict_keys(['receptor', 'ligands', 'npndes', 'pockets', 'apo_structures', 'pred_structures', 'system_annotation'])


In [16]:
result["receptor"]

StructureData(coords=array([[48.761, 52.015, 58.719],
       [48.178, 50.968, 57.827],
       [49.186, 50.593, 56.712],
       ...,
       [50.066, 42.485, 54.198],
       [47.368, 43.433, 55.544],
       [50.022, 44.674, 54.248]], dtype=float32), atom_names=array(['N', 'CA', 'C', ..., 'O', 'CB', 'OXT'], dtype='<U6'), res_ids=array([  2,   2,   2, ..., 315, 315, 315]), res_names=array(['SER', 'SER', 'SER', ..., 'ALA', 'ALA', 'ALA'], dtype='<U5'), chain_ids=array(['A', 'A', 'A', ..., 'A', 'A', 'A'], dtype='<U4'), cif='systems/4agi__1__1.C__1.W/receptor.cif')

In [17]:
result["ligands"]

{'1.W': LigandData(coords=array([[48.954, 28.186, 62.068],
        [48.681, 29.643, 61.777],
        [48.556, 29.933, 60.282],
        [47.499, 28.988, 59.699],
        [47.948, 27.56 , 59.939],
        [46.992, 26.503, 59.374],
        [49.72 , 30.41 , 62.383],
        [48.255, 31.352, 60.148],
        [46.262, 29.193, 60.366],
        [48.031, 27.383, 61.339],
        [50.647, 25.895, 62.426],
        [50.853, 27.716, 61.696]]), atom_types=array([ 0,  0,  0,  0,  0,  0,  3,  3,  3,  3,  0, 10]), atom_charges=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), bond_types=array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32), bond_indices=array([[ 0,  1],
        [ 0,  9],
        [ 0, 11],
        [ 1,  2],
        [ 1,  6],
        [ 2,  3],
        [ 2,  7],
        [ 3,  4],
        [ 3,  8],
        [ 4,  5],
        [ 4,  9],
        [10, 11]]), sdf='systems/4agi__1__1.C__1.W/ligand_files/1.W.sdf')}

In [18]:
result["npndes"]

In [19]:
result["pockets"]

{'1.W': StructureData(coords=array([[44.328, 25.255, 69.015],
        [44.807, 23.896, 68.768],
        [45.221, 23.236, 70.094],
        [45.184, 23.857, 71.179],
        [45.951, 23.832, 67.734],
        [47.212, 24.557, 68.172],
        [47.414, 24.863, 69.344],
        [48.076, 24.809, 67.215],
        [36.382, 28.311, 60.748],
        [36.617, 26.986, 60.126],
        [37.223, 26.069, 61.159],
        [37.742, 26.535, 62.179],
        [37.517, 27.02 , 58.904],
        [38.767, 27.825, 59.216],
        [36.781, 27.579, 57.685],
        [39.863, 27.657, 58.228],
        [41.34 , 21.613, 65.357],
        [41.922, 22.891, 65.034],
        [40.765, 23.843, 64.71 ],
        [39.823, 23.451, 64.019],
        [42.844, 22.769, 63.836],
        [44.24 , 22.457, 64.251],
        [44.481, 21.251, 64.558],
        [45.042, 23.432, 64.389],
        [46.041, 30.034, 52.187],
        [47.128, 29.485, 52.997],
        [47.411, 28.073, 52.536],
        [46.588, 27.153, 52.689],
        [46.793, 29.

In [20]:
result["apo_structures"]

{'4uou_B': StructureData(coords=array([[48.429, 51.569, 59.041],
        [47.897, 50.834, 57.841],
        [48.919, 50.677, 56.677],
        ...,
        [50.148, 42.882, 53.814],
        [47.726, 43.263, 55.61 ],
        [49.977, 45.05 , 54.02 ]], dtype=float32), atom_names=array(['N', 'CA', 'C', ..., 'O', 'CB', 'OXT'], dtype='<U6'), res_ids=array([  1,   1,   1, ..., 314, 314, 314]), res_names=array(['SER', 'SER', 'SER', ..., 'ALA', 'ALA', 'ALA'], dtype='<U5'), chain_ids=array(['B', 'B', 'B', ..., 'B', 'B', 'B'], dtype='<U4'), cif='linked_structures/apo/4agi__1__1.C__1.W/4uou_B/superposed.cif'),
 '4uou_C': StructureData(coords=array([[48.891, 51.823, 58.85 ],
        [48.219, 50.988, 57.793],
        [49.153, 50.808, 56.609],
        ...,
        [49.93 , 42.166, 53.822],
        [47.663, 43.299, 55.58 ],
        [50.202, 44.363, 53.803]], dtype=float32), atom_names=array(['N', 'CA', 'C', ..., 'O', 'CB', 'OXT'], dtype='<U6'), res_ids=array([  1,   1,   1, ..., 314, 314, 314]), res_na

In [21]:
result["pred_structures"]

{'Q4WW81_A': StructureData(coords=array([[49.542, 52.802, 62.086],
        [49.671, 53.148, 60.651],
        [49.148, 51.981, 59.823],
        ...,
        [49.97 , 42.45 , 54.315],
        [47.126, 43.54 , 55.386],
        [49.955, 44.676, 54.549]], dtype=float32), atom_names=array(['N', 'CA', 'C', ..., 'O', 'CB', 'OXT'], dtype='<U6'), res_ids=array([  1,   1,   1, ..., 315, 315, 315]), res_names=array(['MET', 'MET', 'MET', ..., 'ALA', 'ALA', 'ALA'], dtype='<U5'), chain_ids=array(['A', 'A', 'A', ..., 'A', 'A', 'A'], dtype='<U4'), cif='linked_structures/pred/4agi__1__1.C__1.W/Q4WW81_A/superposed.cif')}

In [22]:
result["system_annotation"]

{'pdb_id': '4agi',
 'biounit_id': '1',
 'ligands': [{'pdb_id': '4agi',
   'biounit_id': '1',
   'asym_id': 'W',
   'instance': 1,
   'ccd_code': 'SFU',
   'plip_type': 'SMALLMOLECULE',
   'bird_id': '',
   'centroid': [49.18450164794922, 28.385324478149414, 61.18277359008789],
   'smiles': 'C[Se][C@@H]1O[C@@H](C)[C@@H](O)[C@@H](O)[C@@H]1O',
   'resolved_smiles': 'C[Se][C@@H]1O[C@@H](C)[C@H]([C@H]([C@@H]1O)O)O',
   'residue_numbers': [1],
   'rdkit_canonical_smiles': 'C[Se][C@@H]1O[C@@H](C)[C@@H](O)[C@@H](O)[C@@H]1O',
   'molecular_weight': 242.005730228,
   'crippen_clogp': -1.4338000000000004,
   'num_rot_bonds': 1,
   'num_hbd': 3,
   'num_hba': 4,
   'num_rings': 1,
   'num_heavy_atoms': 12,
   'is_covalent': False,
   'covalent_linkages': [],
   'neighboring_residues': {'1.C': [22,
     23,
     24,
     40,
     44,
     271,
     280,
     282,
     284,
     296,
     298,
     300,
     308,
     309]},
   'neighboring_ligands': [],
   'interacting_residues': {'1.C': [282, 44, 

In [23]:
print(result["system_annotation"]["ligands"][0]['is_artifact'])

False
